# A3: A\*, IDS, and Effective Branching Factor

For this assignment, I implemented the Recursive Best-First Search implementation of the A\* algorithm given in class.  Also in this notebook I included my `iterativeDeepeningSearch` functions from previous assignments.  Below is also a new function named `ebf` that returns an estimate of the effective branching factor for a search algorithm applied to a search problem.

So, the required functions I defined are

   - `aStarSearch(startState, actionsF, takeActionF, goalTestF, hF)`
   - `iterativeDeepeningSearch(startState, goalState, actionsF, takeActionF, maxDepth)`
   - `ebf(nNodes, depth, precision=0.01)`, returns number of nodes expanded and depth reached during a search.

I applied `iterativeDeepeningSearch` and `aStarSearch` to several eight-tile sliding puzzle problems. To do this, I have included my functions from Assignment 2 below

  * `actionsF_8p(state)`: returns a list of up to four valid actions that can be applied in `state`. With each action include a step cost of 1. For example, if all four actions are possible from this state, return [('left', 1), ('right', 1), ('up', 1), ('down', 1)].
  * `takeActionF_8p(state, action)`: return the state that results from applying `action` in `state` and the cost of the one step,
  
plus the following function for the eight-tile puzzle:

  * `goalTestF_8p(state, goal)`
  
Below, I applied `iterativeDeepeningSearch` and `aStarSearch` to several eight-tile sliding puzzle problems and compared their results by displayng solution path depth, number of nodes generated, and the effective branching factor (see discusison of results below for more info). This comparison is done by defining the following function that prints the table as shown in the example below.

   - runExperiment(goalState1, goalState2, goalState3, [h1, h2, h3])

## Node

I will be using a Node class in Python to help store and compare the $f$, $g$, and $h$ values of the various nodes that I am expanding

In [350]:
class Node:
    def __init__(self, state, f=0, g=0 ,h=0):
        self.state = state
        self.f = f
        self.g = g
        self.h = h
    def __repr__(self):
        return "Node(" + repr(self.state) + ", f=" + repr(self.f) + \
               ", g=" + repr(self.g) + ", h=" + repr(self.h) + ")"

## functions

This is the iterative deepening search that I have implemented in previous assignments. Given a startState, goalState, actions function, takeActions function, and a max depth, find the various successor states by applying takeActions function against actions generated from actions function. If goal state exists before cutoff occurs, return that state. Otherwise return failure.

In [351]:
def iterativeDeepeningSearch(startState, goalState, actionsF, takeActionF, maxDepth):
    """ takes in a maxDepth and works its way up to that depth, calling into
    depthLimitedSearch along the way"""
    global nNodes
    nNodes = 0
    
    for depth in range(maxDepth):
        result = depthLimitedSearch(startState, goalState, actionsF, takeActionF, depth)
        if result is 'failure':
            return 'failure'
        if result is not 'cutoff':
            # Add startState to front of solution path, in result, returned by depthLimitedSearch       
            return [startState] + result
    return 'cutoff'

In [352]:
def depthLimitedSearch(state, goalState, actionsF, takeActionF, depthLimit):
    """given a depth, recursively checks the tree to find child states when running the 
    possible actions generated by actionsF through takeActionF"""
    
    if (state == goalState):
        return []
    
    if (depthLimit == 0):
        return 'cutoff' # signal that the depth limit was reached
    
    cutoffOccurred = False
    for action in actionsF(state):
        global nNodes
        nNodes += 1
        childState = takeActionF(state, action)[0]
        result = depthLimitedSearch(childState, goalState, actionsF, takeActionF, depthLimit-1)
        
        if result == 'cutoff':
            cutoffOccurred = True
        
        elif result != 'failure':
            # Add childState to front of partial solution path, in result, returned by depthLimitedSearch
            return [childState] + result
    if cutoffOccurred:
        return 'cutoff'
    else:
        return 'failure'

8p functions from previous assignment, these will be used to see how aStarSearch solves the problem using these same helper functions. Notice that the goalTest is now a function to ensure more complex goals can be found.

Also, the return of actionsF_8p has changed slightly to return a step cost (always 1)

In [353]:
def findBlank_8p(state):
    """returns the row and column index of the blank (0 value) as tuple (row, column)"""
    blankIndex = state.index(0)
    if blankIndex > 2 and blankIndex < 6:
        idx1 = 1
        idx2 = blankIndex - 3
        
    elif blankIndex > 5:
        idx1 = 2
        idx2 = blankIndex - 6
        
    else:
        idx1 = 0
        idx2 = blankIndex

    return (idx1, idx2)

In [354]:
def actionsF_8p(state): 
    """generates the list of actions available for the blank ie where it can move to"""

    blankState = findBlank_8p(state)
    blank = (blankState[0] * 3) + blankState[1]
    
    actions = []
    if blank > 0:
        actions.append(('left', 1))
        
    if blank < 8:
        actions.append(('right', 1))
        
    if blankState[0] > 0:
        actions.append(('up', 1))

    if blankState[0] < 2:
        actions.append(('down', 1))
    
    return actions

In [355]:
import copy

def takeActionF_8p(state, action):
    """ given the state of the board, adjust based on the 
    action given to return a new state"""
    
    blankState = findBlank_8p(state)
    blank = (blankState[0] * 3) + blankState[1]
    
    if (action[0] == 'down'):
        belowBlank = blank + 3
        newState = copy.copy(state)
        
        newState[belowBlank], newState[blank] = newState[blank], newState[belowBlank]
        return (newState, action[1])
        
    if (action[0] == 'up'):
        aboveBlank = blank - 3
        newState = copy.copy(state)
        
        newState[aboveBlank], newState[blank] = newState[blank], newState[aboveBlank]
        return (newState, action[1])
        
    if (action[0] == 'left'):
        lefOfBlank = blank - 1
        newState = copy.copy(state)
        
        newState[lefOfBlank], newState[blank] = newState[blank], newState[lefOfBlank]
        return (newState, action[1])
    
    if (action[0] == 'right'):
        rightOfBlank = blank + 1
        newState = copy.copy(state)
        
        newState[rightOfBlank], newState[blank] = newState[blank], newState[rightOfBlank]
        return (newState, action[1])
        
    # default
    return state

In [356]:
def goalTestF_8p(state, goal):
    return state == goal

examples of 8 puzzle functions:

In [357]:
startState = [1, 0, 2, 3, 4, 5, 6, 7, 8]

In [358]:
findBlank_8p(startState)

(0, 1)

In [359]:
actionsF_8p(startState)

[('left', 1), ('right', 1), ('down', 1)]

In [360]:
takeActionF_8p(startState, ('down', 1)) # move the blank down, when shown in list this means adding 3 to the index of 0

([1, 4, 2, 3, 0, 5, 6, 7, 8], 1)

In [361]:
goalState = [1, 0, 2, 3, 4, 5, 6, 7, 8]
goalTestF_8p(startState, goalState)

True

So, now I have a set of functions for the 8-puzzle that I can use. The main difference is the step cost being returned. This is the true step cost of moving any tile. A cost of 1.

I will use this actual step cost when running the comparisons below using different heuristic functions that are guessing at step costs to see what the difference in results is. 

In order to compare though, I need a way to easily determine how much extra work a search may have done given a certain heuristic function. To do that, I can use Effective Branching Factor.

## Heuristic Functions

For `aStarSearch` I used the following two heuristic functions

  * `h1_8p(state, goal)`: $h(state, goal) = 0$, for all states $state$ and all goal states $goal$,
  * `h2_8p(state, goal)`: $h(state, goal) = m$, where $m$ is the Manhattan distance that the blank is from its goal position,
  
I also used one of my own:
  * `h3_8p(state, goal)`: $h(state, goal) = d$, where $d$ is the number of displaced tiles

A heuristic function is estimated cost of the cheapest path from state at node nn to a goal state. I will be using these different functions on the various puzzles to see what results I can get from different values.

In [362]:
def h1_8p(state, goal):
    return 0; # always return 0 as the heuristic

In [363]:
h1_8p([0, 1, 2, 3, 4, 5, 6, 7, 8], [1, 2, 3, 4, 0, 5, 6, 7, 8])

0

It doesn't matter what was entered, just return 0

Manhattan or 'city block' distance will determine a value based on distance without diagonal moves

In [364]:
def h2_8p(state, goal):
    (x_value, y_value) = findBlank_8p(state)
    (x_goal, y_goal) = findBlank_8p(goal)
    
    return abs(x_value - x_goal) + abs(y_value - y_goal)

In [365]:
h2_8p([1, 2, 3, 4, 0, 5, 6, 7, 8], [0, 1, 2, 3, 4, 5, 6, 7, 8]) # one move up, one move over

2

In [366]:
h2_8p([0, 1, 2, 3, 4, 5, 6, 7, 8], [1, 2, 3, 4, 5, 6, 7, 8, 0]) # down 2, over 2

4

My own implementation will find the number of misplaced tiles and use that as the heuristic estimate value.

So if 2 tiles are out of place, return `2`. If 8, return `8`. Straightforward enough!

In [367]:
def h3_8p(state, goal):
    """ find the number of misplaced tiles. check against the goalstate
    tile index to see if it is the same. if not, add to the misplaced count"""
    
    misplaced = 0
    for (item, index) in enumerate(state):
        if (item != goal[index]):
            misplaced += 1
    
    return misplaced

In [368]:
h3_8p([0, 1, 2, 3, 4, 5, 6, 7, 8], [1, 0, 2, 3, 4, 5, 6, 7, 8]) # 2 out of place

2

In [369]:
h3_8p([0, 1, 2, 3, 4, 5, 6, 7, 8], [1, 2, 3, 4, 5, 6, 7, 8, 0]) # all 9 out of place

9

## Effective branching factor

This is the function for calculating effective branching factor.

The branching factor of a tree is the number of children at each node. If this is not the same for all nodes, then we can find an average, or "effective" branching factor.

To find this, `ebf` uses a binary search. It will use a low and high value of $b$ that brackets the true value. For a search that explored $n$ nodes to a maximum depth of $d$,

 $$ \frac{1-b^{d+1}}{1-b}$$
 
 
This was pretty confusing to me, so let me try and explain what is happening in the following function. I need to figure out what $b$ is in the above equation, also written as $1 + b + b^2 + b^3 ... + b^d$ where $d$ is the depth. Basically, if I search down a tree of depth 3, how many branches can I guess to be the "typical" number of successors a node had if I know I expanded 4 nodes total?

If I found the 'average' successors, that number would have to grow exponentially because each child would have that same number of successors. So the above equations show that number growing up until $b^d$ times. The best way to solve this is to, essentially, guess by using a binary search. I know the number cannot be greater than the number of nodes, or less than 1, so I will search in between those two values until I find a number that is close enough to the number of nodes expanded.

In [370]:
def ebf(nNodes, depth, precision=0.01):  
    """ calculates the effective branching factor which is the effective number of children
    expanded at each node. this is calculated as a result of the number of nodes and the depth
    searched -- see above for the equation """
    
    first = 1
    last = nNodes
    found = False
    midpoint = float("inf")
    
    if depth == 0:
        return nNodes
    
    while first<=last and not found:
        midpoint = (first + last)/2
        answer = (1 - (midpoint ** (depth + 1))) / (1 - midpoint)

        if abs(nNodes - answer) < precision:
            found = True
        else:
            if nNodes < answer:
                last = midpoint
            else:
                first = midpoint

    return midpoint

In [371]:
ebf(10, 3)

1.661376953125

The smallest argument values should be a depth of 0, and 1 node.

In [372]:
ebf(1, 0)

1

In [373]:
ebf(2, 1)

1.0078125

In [374]:
ebf(2, 1, precision=0.000001)

1.0000009536743164

In [375]:
ebf(200000, 5)

11.275596931956898

In [376]:
ebf(200000, 50)

1.2348192492705223

Here is a simple example using our usual simple graph search.

In [377]:
def actionsF_simple(state):
    succs = {'a': ['b', 'c'], 'b':['a'], 'c':['h'], 'h':['i'], 'i':['j', 'k', 'l'], 'k':['z']}
    return [(s, 1) for s in succs.get(state, [])]

def takeActionF_simple(state, action):
    return action

def goalTestF_simple(state, goal):
    return state == goal

def h_simple(state, goal):
    return 1

In [378]:
actions = actionsF_simple('a')
actions

[('b', 1), ('c', 1)]

In [379]:
takeActionF_simple('a', actions[0])

('b', 1)

In [380]:
goalTestF_simple('a', 'a')

True

In [381]:
h_simple('a', 'z')

1

## simple

Okay! Let's give it a go, first with `iterativeDeepeningSearch` and then with `aStarSearch`

In [382]:
iterativeDeepeningSearch('a', 'z', actionsF_simple, takeActionF_simple, 10)

['a', 'c', 'h', 'i', 'k', 'z']

aStarSearch will take a startState, actions function, and take actions function. It will also take a goalTest function (passed as a lambda) and a heuristic function (also passed as lamda)

In [383]:
nNodes = 0

I defined a global `nNodes` to keep track of the number of exapnded nodes

In [384]:
def exists(child, children):
    """ check if item is in children, I thought this might help with run times
    but it seems to have no effect"""
    
    for item in children:
        if(item.state == child.state):
            return True
    return False

In [385]:
exists(Node('k', f=6, g=4, h=1), [Node('k', f=6, g=4, h=1)])

True

In [386]:
def aStarSearch(startState, actionsF, takeActionF, goalTestF, hF):
    """ starts out the aStarSearch by calling into aStarSearchHelper
    creates the initial node with a startstate and step value of 0"""
    global nNodes
    nNodes = 0
    h = hF(startState)
    startNode = Node(state=startState, f=0+h, g=0, h=h)
    return aStarSearchHelper(startNode, actionsF, takeActionF, goalTestF, hF, float('inf'))

def aStarSearchHelper(parentNode, actionsF, takeActionF, goalTestF, hF, fmax):
    """ helperfunction invoked by aStarSearch generates a list of children
    based of the actions available, and recurseively applies an f value based off of
    the heuristic value plus the step cost"""
    
#    from IPython.core.debugger import Tracer; Tracer()()
    global nNodes
    if goalTestF(parentNode.state):
        return ([parentNode.state], parentNode.g)
    
    ## Construct list of children nodes with f, g, and h values
    actions = actionsF(parentNode.state)
    if not actions:
        return ("failure", float('inf'))
    children = []
    for action in actions:
        nNodes += 1
        (childState,stepCost) = takeActionF(parentNode.state, action)
        h = hF(childState)
        g = parentNode.g + stepCost
        f = max(h+g, parentNode.f)
        childNode = Node(state=childState, f=f, g=g, h=h)
        
        # append child if it doesn't exist already
        if (not exists(childNode, children)):
            children.append(childNode)
        
    while True: 
        # find best child
        children.sort(key = lambda n: n.f) # sort by f value
        bestChild = children[0]

        if bestChild.f > fmax:
            return ("failure",bestChild.f)
        
        # next lowest f value
        alternativef = children[1].f if len(children) > 1 else float('inf')
        
        # expand best child, reassign its f value to be returned value
        result,bestChild.f = aStarSearchHelper(bestChild, actionsF, takeActionF, goalTestF, hF, min(fmax,alternativef))
        if result is not "failure":
            result.insert(0,parentNode.state) 
            return (result, bestChild.f)    

In [387]:
aStarSearch('a',actionsF_simple, takeActionF_simple,
            lambda s: goalTestF_simple(s, 'z'),
            lambda s: h_simple(s, 'z'))

(['a', 'c', 'h', 'i', 'k', 'z'], 5)

Cool! Same result but we can also get a step count back that will adjust based on the heuristic funtions guess at what is the best way to go!

In [388]:
aStarSearch('2',actionsF_simple, takeActionF_simple, # fails due to never hitting that goal
            lambda s: goalTestF_simple(s, 'z'),
            lambda s: h_simple(s, 'z'))

('failure', inf)

Excellent. Let's add a more difficult puzzle and see what happens. This is where the other functions from above come into play. I will be using the 8-puzzle example with aStar search and the three different heuristic functions. The third is the displaced tile function that I chose. I will use `runExperiment` to check the 3 different goal states against the list of heuristic functions I pass in

In [389]:
goalState = [1, 2, 3, 4, 0, 5, 6, 7, 8]
startState = [1, 2, 3, 4, 5, 8, 6, 0, 7]

aStarSearch(startState, actionsF_8p, takeActionF_8p, 
            lambda s: goalTestF_8p(s, goalState), lambda s: h1_8p(s, goalState))

([[1, 2, 3, 4, 5, 8, 6, 0, 7],
  [1, 2, 3, 4, 5, 8, 6, 7, 0],
  [1, 2, 3, 4, 5, 0, 6, 7, 8],
  [1, 2, 3, 4, 0, 5, 6, 7, 8]],
 3)

In [390]:
goalState = [1, 0, 3, 4, 5, 8, 2, 6, 7] 
startState = [1, 2, 3, 4, 0, 5, 6, 7, 8]
aStarSearch(startState, actionsF_8p, takeActionF_8p, 
            lambda s: goalTestF_8p(s, goalState), lambda s: h1_8p(s, goalState))

([[1, 2, 3, 4, 0, 5, 6, 7, 8],
  [1, 0, 3, 4, 2, 5, 6, 7, 8],
  [0, 1, 3, 4, 2, 5, 6, 7, 8],
  [4, 1, 3, 0, 2, 5, 6, 7, 8],
  [4, 1, 3, 2, 0, 5, 6, 7, 8],
  [4, 1, 3, 2, 5, 0, 6, 7, 8],
  [4, 1, 3, 2, 5, 8, 6, 7, 0],
  [4, 1, 3, 2, 5, 8, 6, 0, 7],
  [4, 1, 3, 2, 5, 8, 0, 6, 7],
  [4, 1, 3, 0, 5, 8, 2, 6, 7],
  [0, 1, 3, 4, 5, 8, 2, 6, 7],
  [1, 0, 3, 4, 5, 8, 2, 6, 7]],
 11)

In [391]:
goalState = [1, 0, 3, 4, 5, 8, 2, 6, 7] 
startState = [1, 2, 3, 4, 0, 5, 6, 7, 8]
aStarSearch(startState, actionsF_8p, takeActionF_8p, 
            lambda s: goalTestF_8p(s, goalState), lambda s: h3_8p(s, goalState))

print(nNodes)

4350763


In [392]:
iterativeDeepeningSearch(startState, goalState, actionsF_8p, takeActionF_8p, 15)
print(nNodes)

864543


I will apply all four algorithms (`iterativeDeepeningSearch` plus `aStarSearch` with the three heuristic functions) to three eight-tile puzzle problems with start state

$$
\begin{array}{ccc}
1 & 2 & 3\\
4 & 0 & 5\\
6 & 7 & 8
\end{array}
$$

and these three goal states.

$$
\begin{array}{ccccccccccc}
1 & 2 & 3  & ~~~~ & 1 & 2 & 3  &  ~~~~ & 1 & 0 &  3\\
4 & 0 & 5  & & 4 & 5 & 8  & & 4 & 5 & 8\\
6 & 7 & 8 &  & 6 & 0 & 7  & & 2 & 6 & 7
\end{array}
$$

In [393]:
import pandas

def runExperiment(goalState1, goalState2, goalState3, heuristicsFuntions):
    """ takes in the three goal states and runs them against the list of heuristic functions
    also runs the search with iterativeDeepeningSearch"""
    
    goals = [goalState1, goalState2, goalState3]
    data = []
    ids = []
    
    print('         ', goalState1, ' ', goalState2, '   ', goalState3)
    print('Algorithm    Depth  Nodes  EBF              Depth  Nodes  EBF              Depth   Nodes     EBF')
    for goal in goals:
        for h in heuristicsFuntions:
            (run, depth) = aStarSearch(startState, actionsF_8p, takeActionF_8p, 
                             lambda s: goalTestF_8p(s, goal), lambda s: h(s, goal))
            
            data.append([depth, nNodes, "{0:.3f}".format(ebf(nNodes, depth))])
        
        resultIds = iterativeDeepeningSearch(startState, goal, actionsF_8p, takeActionF_8p, 20)
        depthIds = len(resultIds) - 1
        formatIds = [depthIds, nNodes, "{0:.3f}".format(ebf(nNodes, depthIds))]
        ids.append(formatIds)
        
    # wasted a lot of time trying to understand the python string format minilanguage 
    # and dataframes so I'm just doing it manually for now
    print('IDS',  '          ', ids[0][0], '    ', ids[0][1], ' ', ids[0][2], '            ', ids[1][0], '   ', ids[1][1], '   ', ids[1][2], '            ', ids[2][0], '  ', ids[2][1], '  ', ids[2][2])
    print('A*h1', '         ', data[0][0], '    ', data[0][1], ' ', data[0][2], '            ', data[3][0], '   ', data[3][1], '  ', data[3][2], '            ', data[6][0], ' ', data[6][1], '  ', data[6][2])
    print('A*h2', '         ', data[1][0], '    ', data[1][1], ' ', data[1][2], '            ', data[4][0], '    ', data[4][1], '  ', data[4][2], '            ', data[7][0], '  ', data[7][1], '  ', data[7][2])
    print('A*h3', '         ', data[2][0], '    ', data[2][1], ' ', data[2][2], '            ', data[5][0], '   ', data[5][1], '  ', data[5][2], '            ', data[8][0], ' ', data[8][1], '  ', data[8][2])

In [394]:
goalState1 = [1, 2, 3, 4, 0, 5, 6, 7, 8]
goalState2 = [1, 2, 3, 4, 5, 8, 6, 0, 7]
goalState3 = [1, 0, 3, 4, 5, 8, 2, 6, 7]

runExperiment(goalState1, goalState2, goalState3, [h1_8p, h2_8p, h3_8p])

          [1, 2, 3, 4, 0, 5, 6, 7, 8]   [1, 2, 3, 4, 5, 8, 6, 0, 7]     [1, 0, 3, 4, 5, 8, 2, 6, 7]
Algorithm    Depth  Nodes  EBF              Depth  Nodes  EBF              Depth   Nodes     EBF
IDS            0      0   0.000              3     55     3.394              11    864543    3.355
A*h1           0      0   0.000              3     175    5.212              11   2877586    3.758
A*h2           0      0   0.000              3      61    3.531              11    226717    2.955
A*h3           0      0   0.000              3     280    6.169              11   4350763    3.907


## conclusion

My heuristic function is admissible. It never overestimates because being out of position $might$ mean that the tile needs to move $more$ than the one "point" that I am giving that out-of-place tile. It will never be less than that, since it is out of place, it needs to move at least once.

`h2` was the best heuristic function that expanded fewer nodes resulting in a better EBF than the other runs. I was surprised to see IDS getting better results than some of the A\* searches.

It was hard to come up with my own function and I ended up using one that we have discussed in the past. I think this was the 2nd hardest thing for me to wrap my head around (1st being the `ebf` function). I am not sure how anyone would come up with a heuristic function -- it seemed to take a decent leap of understanding that I seem not to posess.